In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.applications import VGG16, MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

In [ ]:
agg_train_1 = np.load(r'/content/drive/My Drive/Baby Cry/Agg feat strat k fold data/agg_train_1.npy')
agg_test_1 = np.load(r'/content/drive/My Drive/Baby Cry/Agg feat strat k fold data/agg_test_1.npy')


agg_train_1 = np.reshape(agg_train_1,(agg_train_1.shape[0],agg_train_1.shape[1]*agg_train_1.shape[2]))
agg_test_1 = np.reshape(agg_test_1,(agg_test_1.shape[0],agg_test_1.shape[1]*agg_test_1.shape[2]))

print(agg_train_1.shape)
print(agg_test_1.shape)

(605, 136)
(152, 136)


In [ ]:
train_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/train_1.npy')
test_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/test_1.npy')
train_lab_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/train_lab_1.npy')
test_lab_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/test_lab_1.npy')


baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(288,432, 3)))
# add layers for transfer learning
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4),name = 'averagepool_last')(headModel)
headModel = Flatten(name="flatten")(headModel)


tensor_train = Input(shape=(agg_train_1.shape[1],))
merged_output = tf.keras.layers.Concatenate(axis=1)([headModel, tensor_train])

x = Dense(512, activation="relu",name = 'MLP_512')(merged_output)
x = Dense(128, activation="relu",name = 'MLP_128')(x)   # we can experiment with more dense layers if output of average pooling layer is too big
x = Dropout(0.3,name = 'dropout_MLP')(x)
out_mlp = Dense(5, activation="softmax",name = 'output')(x)  # we have 5 classes

#build the mlp model
total_model = Model(inputs = [baseModel.input,tensor_train], outputs=out_mlp)

opt=Adam(lr=1e-4)
total_model.compile(loss='categorical_crossentropy', optimizer=opt,	metrics=["accuracy"])
total_model.summary()


#headModel = Dense(1024, activation="relu")(headModel)
"""
headModel = Dense(512, activation="relu",name = 'dense_512')(headModel)
headModel = Dense(128, activation="relu",name = 'dense_128')(headModel)   # we can experiment with more dense layers if output of average pooling layer is too big
headModel = Dropout(0.3,name = 'dropout_last')(headModel)
headModel = Dense(5, activation="softmax",name = 'output')(headModel)  # we have 5 classes
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
#model.summary()
"""
#------------Optional: We can train only Classification layers with below code or do training from scratch with keras.fit()
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
# for layer in baseModel.layers:
#   layer.trainable = False

###for models with Batch-Norm  
for layer in baseModel.layers:
    if layer.__class__.__name__=='BatchNormalization':
      #print('exist')
      layer.trainable = False
    #else:
      #print('normal layer')  

#opt=SGD(learning_rate=1e-4, nesterov=True)
#opt=tf.keras.optimizers.Nadam(learning_rate=1e-4)
"""
opt=Adam(lr=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=opt,	metrics=["accuracy"])
model.summary()
import tensorflow as tf
#callback for best Val Accuracy
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath="/content/best.ckpt",
  save_weights_only=True,
  monitor='val_accuracy',
  mode='max',
  save_best_only=True)
model.fit(train_1, train_lab_1, batch_size=32,	validation_data=(test_1, test_lab_1),	epochs=100, verbose=2, callbacks=[model_checkpoint_callback])
#load the checkpoint from here

#get the latest checkpoint file
checkpoint_dir = os.path.dirname('/content/best.ckpt')
latest = tf.train.latest_checkpoint(checkpoint_dir)

#Create a new model instance
model_latest_checkpoint = model
#model_latest_checkpoint.summary()
# Load the previously saved weights
model_latest_checkpoint.load_weights(latest)
# Re-evaluate the model
loss, acc = model_latest_checkpoint.evaluate(test_1, test_lab_1, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))
"""


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 288, 432, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 288, 432, 64) 1792        input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 288, 432, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 144, 216, 64) 0           block1_conv2[0][0]               
_______________________________________________________________________________________

'\nopt=Adam(lr=1e-4)\nmodel.compile(loss=\'categorical_crossentropy\', optimizer=opt,\tmetrics=["accuracy"])\nmodel.summary()\nimport tensorflow as tf\n#callback for best Val Accuracy\nmodel_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(\n  filepath="/content/best.ckpt",\n  save_weights_only=True,\n  monitor=\'val_accuracy\',\n  mode=\'max\',\n  save_best_only=True)\nmodel.fit(train_1, train_lab_1, batch_size=32,\tvalidation_data=(test_1, test_lab_1),\tepochs=100, verbose=2, callbacks=[model_checkpoint_callback])\n#load the checkpoint from here\n\n#get the latest checkpoint file\ncheckpoint_dir = os.path.dirname(\'/content/best.ckpt\')\nlatest = tf.train.latest_checkpoint(checkpoint_dir)\n\n#Create a new model instance\nmodel_latest_checkpoint = model\n#model_latest_checkpoint.summary()\n# Load the previously saved weights\nmodel_latest_checkpoint.load_weights(latest)\n# Re-evaluate the model\nloss, acc = model_latest_checkpoint.evaluate(test_1, test_lab_1, verbose=2)\nprint

In [ ]:
# adding regularization
regularizer = tf.keras.regularizers.l2(.0001)

for layer in total_model.layers:
    if layer.trainable == True:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)



In [ ]:
import tensorflow as tf
#callback for best Val Accuracy
new_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/total_best.ckpt",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
#total_model
total_model.fit([train_1,agg_train_1], train_lab_1, batch_size=32,	validation_data=([test_1,agg_test_1], test_lab_1),	epochs=20, verbose=2, callbacks=[new_model_checkpoint_callback])

Epoch 1/20


In [ ]:
#load the checkpoint from here

#get the latest checkpoint file
checkpoint_dir = os.path.dirname('/content/total_best.ckpt')
latest = tf.train.latest_checkpoint(checkpoint_dir)

#Create a new model instance
model_latest_checkpoint = total_model
#model_latest_checkpoint.summary()
# Load the previously saved weights
model_latest_checkpoint.load_weights(latest)
# Re-evaluate the model
loss, acc = model_latest_checkpoint.evaluate([test_1,agg_test_1],  test_lab_1, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


5/5 - 1s - loss: 1.6111 - accuracy: 0.7171
Restored model, accuracy: 71.71%
